## Projeto 1 – Backtest Hello World


1.Usando o mesmo grupo dos APS’s

2.Escolha um universo de ativos (>5) em um período t de 

3.Construa uma estratégia de tendência “simples” ou reversão à média (como, mas não pode ser exatamente, um
cruzamento de SMA)

4.Construa a lógica de alocação e backtesting da 

5.Realize testes de sensibilidade no tempo e dos parâmetros

6.Desenvolva uma versão mais “complexa” dessa estratégia

7.Construa a lógica de alocação e backtesting dessa versão mais complexa

8.Realize testes de sensibilidade no tempo e dos parâmetros

9.Apresente em sala os resultados da estratégia (os resultados NÃO precisam ser positivos!!! O importante é tentar)

1.Esta apresentação em sala não requer um ppt, apenas uma passagem pelo código, e uma demonstração dos
resultados. (Em tese a organização do código junto dos resultados deve ser suficiente para o acompanhamento da
estratégia)
10.Durante a apresentação é esperada uma participação de todos os alunos em sala, trazendo dúvidas, questionamentos e
sugestões

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_data(file_path):
    """Loads data from a JSON file and converts to DataFrame"""
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.set_index('timestamp')
    return df

def load_all_data(data_folder='data'):
    """Loads all JSON files from the data folder"""
    data = {}
    for file in os.listdir(data_folder):
        if file.endswith('.json'):
            path = os.path.join(data_folder, file)
            # Extract cryptocurrency symbol from the filename
            symbol = file.split('_')[0].replace('usd', '')
            try:
                data[symbol] = load_data(path)
                print(f"Loaded: {symbol} - {len(data[symbol])} records")
            except Exception as e:
                print(f"Error loading {file}: {e}")
    return data

In [ ]:
# Load all data
crypto_data = load_all_data()


In [ ]:
# 2. Create a function to calculate momentum indicators
def calculate_momentum_indicators(df, short_window=5, medium_window=15, long_window=30, rsi_period=14):
    """
    Calculates momentum indicators for a price series
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame with OHLCV data
    short_window : int
        Period for short moving average
    medium_window : int
        Period for medium moving average
    long_window : int
        Period for long moving average
    rsi_period : int
        Period for RSI
        
    Returns:
    --------
    DataFrame with calculated indicators
    """
    df_indicators = df.copy()
    
    # Exponential moving averages
    df_indicators[f'EMA_{short_window}'] = df['close'].ewm(span=short_window, adjust=False).mean()
    df_indicators[f'EMA_{medium_window}'] = df['close'].ewm(span=medium_window, adjust=False).mean()
    df_indicators[f'EMA_{long_window}'] = df['close'].ewm(span=long_window, adjust=False).mean()
    
    # ROC - Rate of Change (Percentage variation)
    df_indicators['ROC_1'] = df['close'].pct_change(1) * 100
    df_indicators['ROC_3'] = df['close'].pct_change(3) * 100
    df_indicators['ROC_5'] = df['close'].pct_change(5) * 100
    
    # RSI - Relative Strength Index
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    
    rs = avg_gain / avg_loss
    df_indicators['RSI'] = 100 - (100 / (1 + rs))
    
    # Pure Momentum - difference between current price and price N periods ago
    df_indicators['Momentum_1'] = df['close'] - df['close'].shift(1)
    df_indicators['Momentum_3'] = df['close'] - df['close'].shift(3)
    df_indicators['Momentum_5'] = df['close'] - df['close'].shift(5)
    
    # MACD - Moving Average Convergence Divergence
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df_indicators['MACD'] = ema_12 - ema_26
    df_indicators['MACD_Signal'] = df_indicators['MACD'].ewm(span=9, adjust=False).mean()
    df_indicators['MACD_Hist'] = df_indicators['MACD'] - df_indicators['MACD_Signal']
    
    # Remove rows with NaN values
    df_indicators = df_indicators.dropna()
    
    return df_indicators

In [ ]:
def simple_momentum_strategy(df, short_window=5, long_window=15, roc_threshold=1.5, rsi_lower=30, rsi_upper=70):
    """
    Implements a simple momentum strategy
    
    Strategy logic:
    - Buy when short EMA crosses above long EMA AND ROC is positive above threshold
    - Sell when short EMA crosses below long EMA OR RSI enters overbought
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame with OHLCV data
    short_window : int
        Period for short moving average
    long_window : int
        Period for long moving average
    roc_threshold : float
        Threshold for ROC to consider positive momentum
    rsi_lower : int
        Lower RSI limit (oversold)
    rsi_upper : int
        Upper RSI limit (overbought)
        
    Returns:
    --------
    DataFrame with strategy signals
    """
    # Calculate indicators
    data = calculate_momentum_indicators(df, short_window=short_window, long_window=long_window)
    
    # Create signals
    data['EMA_Diff'] = data[f'EMA_{short_window}'] - data[f'EMA_{long_window}']
    data['EMA_Diff_Prev'] = data['EMA_Diff'].shift(1)
    
    # Define conditions for signals
    upward_cross = (data['EMA_Diff'] > 0) & (data['EMA_Diff_Prev'] < 0)
    downward_cross = (data['EMA_Diff'] < 0) & (data['EMA_Diff_Prev'] > 0)
    
    strong_momentum = data['ROC_3'] > roc_threshold
    weak_momentum = data['ROC_3'] < -roc_threshold
    
    rsi_overbought = data['RSI'] > rsi_upper
    rsi_oversold = data['RSI'] < rsi_lower
    
    # Create buy and sell signals
    data['Signal'] = 0
    data.loc[upward_cross & strong_momentum, 'Signal'] = 1  # Buy
    data.loc[downward_cross | weak_momentum | rsi_overbought, 'Signal'] = -1  # Sell
    
    # Position: 1 = long, 0 = neutral, -1 = short
    data['Position'] = 0
    current_position = 0
    
    for i in range(len(data)):
        if data['Signal'].iloc[i] == 1 and current_position <= 0:
            current_position = 1  # Buy
        elif data['Signal'].iloc[i] == -1 and current_position >= 0:
            current_position = 0  # Sell/Neutral
        
        data['Position'].iloc[i] = current_position
    
    # For visualization, mark buy and sell points
    data['Buy'] = ((data['Position'] == 1) & (data['Position'].shift(1) == 0))
    data['Sell'] = ((data['Position'] == 0) & (data['Position'].shift(1) == 1))
    
    return data

In [ ]:

def backtest_strategy(df_strategy, btc_data=None, initial_capital=10000, commission=0.001):
    """
    Performs backtesting of the strategy
    
    Parameters:
    -----------
    df_strategy : DataFrame
        DataFrame with strategy signals
    btc_data : DataFrame, optional
        Bitcoin price data for buy & hold comparison
    initial_capital : float
        Initial capital for backtesting
    commission : float
        Commission rate per trade (%)
        
    Returns:
    --------
    DataFrame with backtesting results
    """
    df_backtest = df_strategy.copy()
    
    # Initialize backtesting variables
    df_backtest['Capital'] = initial_capital
    df_backtest['Units'] = 0
    df_backtest['Position_Value'] = 0
    df_backtest['Daily_Return'] = 0
    
    capital = initial_capital
    units = 0
    
    for i in range(1, len(df_backtest)):
        prev_price = df_backtest['close'].iloc[i-1]
        current_price = df_backtest['close'].iloc[i]
        
        # If position changed
        if df_backtest['Buy'].iloc[i]:
            # Sell everything if short (not implemented here) and buy
            units = capital * (1 - commission) / current_price
            capital = 0
            
        elif df_backtest['Sell'].iloc[i]:
            # Sell everything
            capital = units * current_price * (1 - commission)
            units = 0
        
        # Update position value
        position_value = units * current_price
        
        # Record in DataFrame
        df_backtest['Units'].iloc[i] = units
        df_backtest['Position_Value'].iloc[i] = position_value
        df_backtest['Capital'].iloc[i] = capital
        
        # Total equity
        prev_equity = df_backtest['Capital'].iloc[i-1] + df_backtest['Position_Value'].iloc[i-1]
        current_equity = capital + position_value
        
        # Daily return
        if prev_equity > 0:
            df_backtest['Daily_Return'].iloc[i] = (current_equity / prev_equity) - 1
    
    # Calculate performance metrics
    df_backtest['Equity'] = df_backtest['Capital'] + df_backtest['Position_Value']
    df_backtest['Cumulative_Return'] = df_backtest['Equity'] / initial_capital - 1
    
    # Buy & Hold for comparison - for the cryptocurrency itself
    df_backtest['Coin_Buy_Hold_Units'] = initial_capital * (1 - commission) / df_backtest['close'].iloc[0]
    df_backtest['Coin_Buy_Hold_Value'] = df_backtest['Coin_Buy_Hold_Units'] * df_backtest['close']
    df_backtest['Coin_Buy_Hold_Return'] = df_backtest['Coin_Buy_Hold_Value'] / initial_capital - 1
    
    # BTC Buy & Hold for comparison (if provided)
    if btc_data is not None:
        # Reindex BTC data to match the strategy timeframe
        btc_aligned = btc_data.reindex(df_backtest.index, method='ffill')
        
        if not btc_aligned.empty and not btc_aligned['close'].isna().all():
            df_backtest['BTC_Buy_Hold_Units'] = initial_capital * (1 - commission) / btc_aligned['close'].iloc[0]
            df_backtest['BTC_Buy_Hold_Value'] = df_backtest['BTC_Buy_Hold_Units'] * btc_aligned['close']
            df_backtest['BTC_Buy_Hold_Return'] = df_backtest['BTC_Buy_Hold_Value'] / initial_capital - 1
    
    return df_backtest

In [ ]:

def visualize_backtest_results(df_backtest, title="Backtesting Results", short_window=5, long_window=15, btc_comparison=True):
    """
    Visualizes backtesting results
    
    Parameters:
    -----------
    df_backtest : DataFrame
        DataFrame with backtesting results
    title : str
        Chart title
    short_window : int
        Short EMA window used in strategy
    long_window : int
        Long EMA window used in strategy
    btc_comparison : bool
        Whether to include BTC comparison in charts
    """
    fig, axs = plt.subplots(3, 1, figsize=(14, 18), gridspec_kw={'height_ratios': [2, 1, 1]})
    
    # Price and positions chart
    axs[0].plot(df_backtest.index, df_backtest['close'], label='Price', color='gray', alpha=0.6)
    axs[0].plot(df_backtest.index, df_backtest[f'EMA_{short_window}'], label=f'EMA {short_window}', color='blue', alpha=0.7)
    axs[0].plot(df_backtest.index, df_backtest[f'EMA_{long_window}'], label=f'EMA {long_window}', color='red', alpha=0.7)
    
    # Mark buy and sell points
    buys = df_backtest[df_backtest['Buy']]
    sells = df_backtest[df_backtest['Sell']]
    
    axs[0].scatter(buys.index, buys['close'], color='green', s=100, marker='^', label='Buy')
    axs[0].scatter(sells.index, sells['close'], color='red', s=100, marker='v', label='Sell')
    
    axs[0].set_title(f'{title} - Price and Signals')
    axs[0].legend()
    axs[0].grid(True)
    
    # Equity vs Buy & Hold chart
    axs[1].plot(df_backtest.index, df_backtest['Equity'], label='Strategy', color='blue')
    axs[1].plot(df_backtest.index, df_backtest['Coin_Buy_Hold_Value'], label='Coin Buy & Hold', color='orange', alpha=0.7)
    
    # Add BTC comparison if available
    if btc_comparison and 'BTC_Buy_Hold_Value' in df_backtest.columns:
        axs[1].plot(df_backtest.index, df_backtest['BTC_Buy_Hold_Value'], label='BTC Buy & Hold', color='green', alpha=0.7)
    
    axs[1].set_title('Strategy vs Buy & Hold Comparison')
    axs[1].legend()
    axs[1].grid(True)
    
    # Drawdown chart
    axs[2].fill_between(df_backtest.index, 0, df_backtest['DD'] * 100, color='red', alpha=0.3)
    axs[2].set_title('Drawdown (%)')
    axs[2].set_ylim(0, max(100, df_backtest['DD'].max() * 100 * 1.1))
    axs[2].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Display performance metrics
    metrics = calculate_performance_metrics(df_backtest, btc_comparison)
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

In [ ]:

def calculate_performance_metrics(df_backtest, btc_comparison=True):
    """
    Calculates performance metrics for backtesting
    
    Parameters:
    -----------
    df_backtest : DataFrame
        DataFrame with backtesting results
    btc_comparison : bool
        Whether to include BTC comparison metrics
        
    Returns:
    --------
    dict with performance metrics
    """
    # Total return
    total_return = df_backtest['Cumulative_Return'].iloc[-1]
    
    # Annualized return (considering 365 days per year)
    total_days = (df_backtest.index[-1] - df_backtest.index[0]).days
    annualized_return = (1 + total_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    
    # Annualized volatility
    daily_vol = df_backtest['Daily_Return'].std()
    annualized_vol = daily_vol * np.sqrt(365)
    
    # Sharpe Ratio (considering risk-free rate = 0)
    sharpe = annualized_return / annualized_vol if annualized_vol > 0 else 0
    
    # Drawdown
    df_backtest['DD'] = 1 - df_backtest['Equity'] / df_backtest['Equity'].cummax()
    max_drawdown = df_backtest['DD'].max()
    
    # Comparison with Coin Buy & Hold
    coin_bh_total_return = df_backtest['Coin_Buy_Hold_Return'].iloc[-1]
    coin_bh_annualized_return = (1 + coin_bh_total_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    
    # Number of trades
    n_buys = df_backtest['Buy'].sum()
    n_sells = df_backtest['Sell'].sum()
    
    metrics = {
        'Total Return': f"{total_return:.2%}",
        'Annualized Return': f"{annualized_return:.2%}",
        'Annualized Volatility': f"{annualized_vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Maximum Drawdown': f"{max_drawdown:.2%}",
        'Coin Buy & Hold Total Return': f"{coin_bh_total_return:.2%}",
        'Coin Buy & Hold Annualized Return': f"{coin_bh_annualized_return:.2%}",
        'Alpha vs Coin Buy & Hold': f"{total_return - coin_bh_total_return:.2%}",
        'Number of Buys': f"{n_buys}",
        'Number of Sells': f"{n_sells}"
    }
    
    # Add BTC comparison metrics if available
    if btc_comparison and 'BTC_Buy_Hold_Return' in df_backtest.columns:
        btc_bh_total_return = df_backtest['BTC_Buy_Hold_Return'].iloc[-1]
        btc_bh_annualized_return = (1 + btc_bh_total_return) ** (365 / total_days) - 1 if total_days > 0 else 0
        
        metrics.update({
            'BTC Buy & Hold Total Return': f"{btc_bh_total_return:.2%}",
            'BTC Buy & Hold Annualized Return': f"{btc_bh_annualized_return:.2%}",
            'Alpha vs BTC Buy & Hold': f"{total_return - btc_bh_total_return:.2%}"
        })
    
    return metrics

In [ ]:
def parameter_sensitivity_test(df, btc_data=None, parameters=None, initial_capital=10000):
    """
    Performs sensitivity tests for different parameters
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame with OHLCV data
    btc_data : DataFrame, optional
        Bitcoin price data for buy & hold comparison
    parameters : dict
        Dictionary with parameters to be tested
        {parameter: list_of_values}
    initial_capital : float
        Initial capital for backtesting
        
    Returns:
    --------
    DataFrame with test results
    """
    if parameters is None:
        parameters = {
            'short_window': [3, 5],
            'long_window': [10, 15],
            'roc_threshold': [1.5, 2.0],
            'rsi_lower': [30],
            'rsi_upper': [70]
        }
    
    results = []
    
    # Generate all parameter combinations
    short_windows = parameters.get('short_window', [5])
    long_windows = parameters.get('long_window', [15])
    roc_thresholds = parameters.get('roc_threshold', [1.5])
    rsi_lowers = parameters.get('rsi_lower', [30])
    rsi_uppers = parameters.get('rsi_upper', [70])
    
    total_tests = len(short_windows) * len(long_windows) * len(roc_thresholds) * len(rsi_lowers) * len(rsi_uppers)
    print(f"Performing {total_tests} sensitivity tests...")
    
    counter = 0
    for sw in short_windows:
        for lw in long_windows:
            # Ensure short window is shorter than long window
            if sw >= lw:
                continue
                
            for rt in roc_thresholds:
                for rsi_l in rsi_lowers:
                    for rsi_u in rsi_uppers:
                        counter += 1
                        if counter % 10 == 0:
                            print(f"Test {counter}/{total_tests}...")
                        
                        try:
                            # Apply strategy
                            df_strategy = simple_momentum_strategy(
                                df, 
                                short_window=sw,
                                long_window=lw,
                                roc_threshold=rt,
                                rsi_lower=rsi_l,
                                rsi_upper=rsi_u
                            )
                            
                            # Perform backtesting
                            df_backtest = backtest_strategy(df_strategy, btc_data, initial_capital=initial_capital)
                            
                            # Calculate metrics
                            final_return = df_backtest['Cumulative_Return'].iloc[-1]
                            metrics = calculate_performance_metrics(df_backtest)
                            sharpe = float(metrics['Sharpe Ratio'].replace("%", ""))
                            max_dd = df_backtest['DD'].max()
                            coin_bh_return = df_backtest['Coin_Buy_Hold_Return'].iloc[-1]
                            
                            result_entry = {
                                'short_window': sw,
                                'long_window': lw,
                                'roc_threshold': rt,
                                'rsi_lower': rsi_l,
                                'rsi_upper': rsi_u,
                                'total_return': final_return,
                                'sharpe_ratio': sharpe,
                                'max_drawdown': max_dd,
                                'coin_buy_hold_return': coin_bh_return,
                                'alpha_vs_coin': final_return - coin_bh_return
                            }
                            
                            # Add BTC comparison if available
                            if 'BTC_Buy_Hold_Return' in df_backtest.columns:
                                btc_bh_return = df_backtest['BTC_Buy_Hold_Return'].iloc[-1]
                                result_entry['btc_buy_hold_return'] = btc_bh_return
                                result_entry['alpha_vs_btc'] = final_return - btc_bh_return
                            
                            # Record result
                            results.append(result_entry)
                        except Exception as e:
                            print(f"Error in test with parameters sw={sw}, lw={lw}, rt={rt}: {e}")
    
    # Convert to DataFrame and sort by Sharpe Ratio
    df_results = pd.DataFrame(results).sort_values('sharpe_ratio', ascending=False)
    return df_results

In [ ]:
def run_strategy_on_all_cryptos(crypto_data, strategy_params=None, btc_comparison=True):
    """
    Runs the momentum strategy on all cryptocurrencies and compares performance
    
    Parameters:
    -----------
    crypto_data : dict
        Dictionary of cryptocurrency DataFrames
    strategy_params : dict
        Dictionary with strategy parameters
    btc_comparison : bool
        Whether to use BTC as benchmark
        
    Returns:
    --------
    dict with results for all cryptocurrencies
    """
    if strategy_params is None:
        strategy_params = {
            'short_window': 5,
            'long_window': 15,
            'roc_threshold': 1.5,
            'rsi_lower': 30,
            'rsi_upper': 70
        }
    
    # Extract parameters
    short_window = strategy_params.get('short_window', 5)
    long_window = strategy_params.get('long_window', 15)
    roc_threshold = strategy_params.get('roc_threshold', 1.5)
    rsi_lower = strategy_params.get('rsi_lower', 30)
    rsi_upper = strategy_params.get('rsi_upper', 70)
    
    # Get BTC data for comparison if needed
    btc_data = crypto_data.get('btc') if btc_comparison and 'btc' in crypto_data else None
    
    results = {}
    summary = []
    
    print(f"\nRunning momentum strategy on {len(crypto_data)} cryptocurrencies...")
    
    for symbol, data in tqdm(crypto_data.items()):
        if len(data) < max(short_window, long_window) + 10:
            print(f"Skipping {symbol}: insufficient data")
            continue
        
        try:
            # Apply strategy
            crypto_strategy = simple_momentum_strategy(
                data,
                short_window=short_window,
                long_window=long_window,
                roc_threshold=roc_threshold,
                rsi_lower=rsi_lower,
                rsi_upper=rsi_upper
            )
            
            # Perform backtesting - pass BTC data for comparison
            crypto_backtest = backtest_strategy(crypto_strategy, btc_data)
            
            # Store results
            results[symbol] = {
                'strategy': crypto_strategy,
                'backtest': crypto_backtest,
                'metrics': calculate_performance_metrics(crypto_backtest, btc_comparison)
            }
            
            # Add to summary
            metrics = results[symbol]['metrics']
            summary_entry = {
                'symbol': symbol,
                'total_return': float(metrics['Total Return'].replace('%', '')) / 100,
                'annualized_return': float(metrics['Annualized Return'].replace('%', '')) / 100,
                'sharpe_ratio': float(metrics['Sharpe Ratio']),
                'max_drawdown': float(metrics['Maximum Drawdown'].replace('%', '')) / 100,
                'coin_buy_hold_return': float(metrics['Coin Buy & Hold Total Return'].replace('%', '')) / 100,
                'n_trades': int(metrics['Number of Buys'])
            }
            
            # Add BTC comparison if available
            if btc_comparison and 'BTC Buy & Hold Total Return' in metrics:
                summary_entry['btc_buy_hold_return'] = float(metrics['BTC Buy & Hold Total Return'].replace('%', '')) / 100
                summary_entry['alpha_vs_btc'] = float(metrics['Alpha vs BTC Buy & Hold'].replace('%', '')) / 100
            
            summary.append(summary_entry)
            
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(summary).sort_values('sharpe_ratio', ascending=False)
    
    return {
        'detailed_results': results,
        'summary': summary_df
    }

def visualize_multi_crypto_performance(summary_df):
    """
    Visualizes performance across multiple cryptocurrencies
    
    Parameters:
    -----------
    summary_df : DataFrame
        Summary DataFrame with performance metrics for all cryptocurrencies
    """
    # Create figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(16, 14))
    
    # 1. Strategy Returns vs Buy & Hold Returns
    ax 

In [ ]:
def visualize_multi_crypto_performance(summary_df):
    """
    Visualizes performance across multiple cryptocurrencies
    
    Parameters:
    -----------
    summary_df : DataFrame
        Summary DataFrame with performance metrics for all cryptocurrencies
    """
    # Create figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(16, 14))
    
    # 1. Strategy Returns vs Buy & Hold Returns
    ax = axs[0, 0]
    ax.scatter(summary_df['coin_buy_hold_return'], summary_df['total_return'], alpha=0.7)
    
    # Add diagonal line
    max_val = max(summary_df['coin_buy_hold_return'].max(), summary_df['total_return'].max())
    min_val = min(summary_df['coin_buy_hold_return'].min(), summary_df['total_return'].min())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.3)
    
    # Add labels for top and bottom performers
    for i, row in summary_df.nlargest(3, 'total_return').iterrows():
        ax.annotate(row['symbol'], (row['coin_buy_hold_return'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    for i, row in summary_df.nsmallest(3, 'total_return').iterrows():
        ax.annotate(row['symbol'], (row['coin_buy_hold_return'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    ax.set_xlabel('Buy & Hold Return')
    ax.set_ylabel('Strategy Return')
    ax.set_title('Strategy vs Buy & Hold Returns')
    ax.grid(True)
    
    # 2. Return vs Sharpe Ratio
    ax = axs[0, 1]
    ax.scatter(summary_df['sharpe_ratio'], summary_df['total_return'], alpha=0.7)
    
    for i, row in summary_df.nlargest(5, 'sharpe_ratio').iterrows():
        ax.annotate(row['symbol'], (row['sharpe_ratio'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    ax.set_xlabel('Sharpe Ratio')
    ax.set_ylabel('Strategy Return')
    ax.set_title('Return vs Risk-Adjusted Performance')
    ax.grid(True)
    
    # 3. Performance Distribution
    ax = axs[1, 0]
    summary_df['alpha_vs_coin'] = summary_df['total_return'] - summary_df['coin_buy_hold_return']
    
    # Group by performance relative to buy & hold
    outperform = summary_df[summary_df['alpha_vs_coin'] > 0]
    underperform = summary_df[summary_df['alpha_vs_coin'] <= 0]
    
    if 'btc_buy_hold_return' in summary_df.columns:
        summary_df['alpha_vs_btc'] = summary_df['total_return'] - summary_df['btc_buy_hold_return']
        outperform_btc = summary_df[summary_df['alpha_vs_btc'] > 0]
        underperform_btc = summary_df[summary_df['alpha_vs_btc'] <= 0]
        
        ax.bar(['Outperform Own B&H', 'Underperform Own B&H', 'Outperform BTC B&H', 'Underperform BTC B&H'], 
               [len(outperform), len(underperform), len(outperform_btc), len(underperform_btc)],
               color=['green', 'red', 'lightgreen', 'lightcoral'])
    else:
        ax.bar(['Outperform Own B&H', 'Underperform Own B&H'], 
               [len(outperform), len(underperform)],
               color=['green', 'red'])
    
    ax.set_ylabel('Number of Cryptocurrencies')
    ax.set_title('Strategy Performance Distribution')
    
    # 4. Top and Bottom Performers
    ax = axs[1, 1]
    
    # Sort by alpha (strategy return - buy & hold return)
    sorted_df = summary_df.sort_values('alpha_vs_coin', ascending=False)
    
    # Get top 5 and bottom 5
    top_5 = sorted_df.head(5)
    bottom_5 = sorted_df.tail(5)
    
    # Combine and sort for display
    display_df = pd.concat([top_5, bottom_5])
    
    # Colors for bars
    colors = ['green'] * 5 + ['red'] * 5
    
    # Plot horizontal bar chart of alphas
    y_pos = np.arange(len(display_df))
    ax.barh(y_pos, display_df['alpha_vs_coin'], color=colors)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(display_df['symbol'])
    ax.set_xlabel('Alpha (Strategy Return - Buy & Hold Return)')
    ax.set_title('Top and Bottom Performers vs Own Buy & Hold')
    
    # Add vertical line at 0
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nSUMMARY STATISTICS:")
    print(f"Total cryptocurrencies analyzed: {len(summary_df)}")
    print(f"Average strategy return: {summary_df['total_return'].mean():.2%}")
    print(f"Average buy & hold return: {summary_df['coin_buy_hold_return'].mean():.2%}")
    print(f"Average alpha: {summary_df['alpha_vs_coin'].mean():.2%}")
    print(f"Cryptocurrencies outperforming buy & hold: {len(outperform)} ({len(outperform)/len(summary_df):.1%})")
    
    if 'btc_buy_hold_return' in summary_df.columns:
        print(f"Cryptocurrencies outperforming BTC buy & hold: {len(outperform_btc)} ({len(outperform_btc)/len(summary_df):.1%})")
        print(f"BTC buy & hold return: {summary_df.loc[summary_df['symbol'] == 'btc', 'btc_buy_hold_return'].values[0]:.2%}" if 'btc' in summary_df['symbol'].values else "N/A")


In [ ]:
def visualize_multi_crypto_performance(summary_df):
    """
    Visualizes performance across multiple cryptocurrencies
    
    Parameters:
    -----------
    summary_df : DataFrame
        Summary DataFrame with performance metrics for all cryptocurrencies
    """
    # Create figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(16, 14))
    
    # 1. Strategy Returns vs Buy & Hold Returns
    ax = axs[0, 0]
    ax.scatter(summary_df['coin_buy_hold_return'], summary_df['total_return'], alpha=0.7)
    
    # Add diagonal line
    max_val = max(summary_df['coin_buy_hold_return'].max(), summary_df['total_return'].max())
    min_val = min(summary_df['coin_buy_hold_return'].min(), summary_df['total_return'].min())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.3)
    
    # Add labels for top and bottom performers
    for i, row in summary_df.nlargest(3, 'total_return').iterrows():
        ax.annotate(row['symbol'], (row['coin_buy_hold_return'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    for i, row in summary_df.nsmallest(3, 'total_return').iterrows():
        ax.annotate(row['symbol'], (row['coin_buy_hold_return'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    ax.set_xlabel('Buy & Hold Return')
    ax.set_ylabel('Strategy Return')
    ax.set_title('Strategy vs Buy & Hold Returns')
    ax.grid(True)
    
    # 2. Return vs Sharpe Ratio
    ax = axs[0, 1]
    ax.scatter(summary_df['sharpe_ratio'], summary_df['total_return'], alpha=0.7)
    
    for i, row in summary_df.nlargest(5, 'sharpe_ratio').iterrows():
        ax.annotate(row['symbol'], (row['sharpe_ratio'], row['total_return']), 
                   xytext=(5, 5), textcoords='offset points')
    
    ax.set_xlabel('Sharpe Ratio')
    ax.set_ylabel('Strategy Return')
    ax.set_title('Return vs Risk-Adjusted Performance')
    ax.grid(True)
    
    # 3. Performance Distribution
    ax = axs[1, 0]
    summary_df['alpha_vs_coin'] = summary_df['total_return'] - summary_df['coin_buy_hold_return']
    
    # Group by performance relative to buy & hold
    outperform = summary_df[summary_df['alpha_vs_coin'] > 0]
    underperform = summary_df[summary_df['alpha_vs_coin'] <= 0]
    
    if 'btc_buy_hold_return' in summary_df.columns:
        summary_df['alpha_vs_btc'] = summary_df['total_return'] - summary_df['btc_buy_hold_return']
        outperform_btc = summary_df[summary_df['alpha_vs_btc'] > 0]
        underperform_btc = summary_df[summary_df['alpha_vs_btc'] <= 0]
        
        ax.bar(['Outperform Own B&H', 'Underperform Own B&H', 'Outperform BTC B&H', 'Underperform BTC B&H'], 
               [len(outperform), len(underperform), len(outperform_btc), len(underperform_btc)],
               color=['green', 'red', 'lightgreen', 'lightcoral'])
    else:
        ax.bar(['Outperform Own B&H', 'Underperform Own B&H'], 
               [len(outperform), len(underperform)],
               color=['green', 'red'])
    
    ax.set_ylabel('Number of Cryptocurrencies')
    ax.set_title('Strategy Performance Distribution')
    
    # 4. Top and Bottom Performers
    ax = axs[1, 1]
    
    # Sort by alpha (strategy return - buy & hold return)
    sorted_df = summary_df.sort_values('alpha_vs_coin', ascending=False)
    
    # Get top 5 and bottom 5
    top_5 = sorted_df.head(5)
    bottom_5 = sorted_df.tail(5)
    
    # Combine and sort for display
    display_df = pd.concat([top_5, bottom_5])
    
    # Colors for bars
    colors = ['green'] * 5 + ['red'] * 5
    
    # Plot horizontal bar chart of alphas
    y_pos = np.arange(len(display_df))
    ax.barh(y_pos, display_df['alpha_vs_coin'], color=colors)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(display_df['symbol'])
    ax.set_xlabel('Alpha (Strategy Return - Buy & Hold Return)')
    ax.set_title('Top and Bottom Performers vs Own Buy & Hold')
    
    # Add vertical line at 0
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nSUMMARY STATISTICS:")
    print(f"Total cryptocurrencies analyzed: {len(summary_df)}")
    print(f"Average strategy return: {summary_df['total_return'].mean():.2%}")
    print(f"Average buy & hold return: {summary_df['coin_buy_hold_return'].mean():.2%}")
    print(f"Average alpha: {summary_df['alpha_vs_coin'].mean():.2%}")
    print(f"Cryptocurrencies outperforming buy & hold: {len(outperform)} ({len(outperform)/len(summary_df):.1%})")
    
    if 'btc_buy_hold_return' in summary_df.columns:
        print(f"Cryptocurrencies outperforming BTC buy & hold: {len(outperform_btc)} ({len(outperform_btc)/len(summary_df):.1%})")
        print(f"BTC buy & hold return: {summary_df.loc[summary_df['symbol'] == 'btc', 'btc_buy_hold_return'].values[0]:.2%}" if 'btc' in summary_df['symbol'].values else "N/A")


In [ ]:
def main():
    """
    Main function to run the backtesting strategy on all cryptocurrencies
    """
    # Load all data
    crypto_data = load_all_data()
    print(f"Loaded {len(crypto_data)} cryptocurrencies")
    
    # Define strategy parameters
    strategy_params = {
        'short_window': 5,
        'long_window': 15,
        'roc_threshold': 1.5,
        'rsi_lower': 30,
        'rsi_upper': 70
    }
    
    # Run strategy on all cryptocurrencies
    results = run_strategy_on_all_cryptos(crypto_data, strategy_params=strategy_params)
    
    # Visualize overall performance
    print("\nOverall performance across all cryptocurrencies:")
    visualize_multi_crypto_performance(results['summary'])
    
    # Show top 10 performing cryptocurrencies
    print("\nTop 10 cryptocurrencies by Sharpe ratio:")
    top_10 = results['summary'].nlargest(10, 'sharpe_ratio')
    print(top_10[['symbol', 'total_return', 'sharpe_ratio', 'alpha_vs_coin', 'alpha_vs_btc']])
    
    # Visualize top 3 cryptocurrencies in detail
    for symbol in top_10['symbol'].head(3):
        print(f"\nDetailed analysis for {symbol.upper()}:")
        visualize_backtest_results(
            results['detailed_results'][symbol]['backtest'], 
            title=f"{symbol.upper()} - Momentum Strategy",
            short_window=strategy_params['short_window'],
            long_window=strategy_params['long_window']
        )
    
    # Perform parameter sensitivity test for the best performing crypto
    best_symbol = top_10['symbol'].iloc[0]
    print(f"\nPerforming parameter sensitivity test for {best_symbol.upper()}...")
    
    reduced_test_parameters = {
        'short_window': [3, 5, 8],
        'long_window': [10, 15, 20],
        'roc_threshold': [1.0, 1.5, 2.0],
        'rsi_lower': [30],
        'rsi_upper': [70]
    }
    
    sensitivity_results = parameter_sensitivity_test(
        crypto_data[best_symbol],
        btc_data=crypto_data.get('btc'),
        parameters=reduced_test_parameters
    )
    
    print(f"\nBest parameters for {best_symbol.upper()} (top 5):")
    print(sensitivity_results.head(5))
    
    # Visualize parameter sensitivity
    plt.figure(figsize=(16, 10))
    
    # Short Window vs Return
    plt.subplot(2, 2, 1)
    sns.boxplot(x='short_window', y='total_return', data=sensitivity_results)
    plt.title(f'{best_symbol.upper()} - Return by Short Window')
    plt.ylabel('Total Return')
    
    # Long Window vs Return
    plt.subplot(2, 2, 2)
    sns.boxplot(x='long_window', y='total_return', data=sensitivity_results)
    plt.title(f'{best_symbol.upper()} - Return by Long Window')
    
    # ROC Threshold vs Return
    plt.subplot(2, 2, 3)
    sns.boxplot(x='roc_threshold', y='total_return', data=sensitivity_results)
    plt.title(f'{best_symbol.upper()} - Return by ROC Threshold')
    plt.ylabel('Total Return')
    
    # Heatmap: Short Window x Long Window
    plt.subplot(2, 2, 4)
    pivot = sensitivity_results.pivot_table(index='short_window', columns='long_window', values='sharpe_ratio', aggfunc='mean')
    sns.heatmap(pivot, annot=True, cmap='viridis')
    plt.title(f'{best_symbol.upper()} - Sharpe Ratio: Short Window x Long Window')
    
    plt.tight_layout()
    plt.show()


In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
def analyze_aggregate_portfolio_performance(crypto_data, strategy_params=None, initial_allocation=10000, commission=0.001):
    """
    Analyzes the performance of an equally-weighted portfolio of all cryptocurrencies
    using the momentum strategy, and compares it to Bitcoin buy & hold.
    
    Parameters:
    -----------
    crypto_data : dict
        Dictionary of cryptocurrency DataFrames
    strategy_params : dict
        Dictionary with strategy parameters
    initial_allocation : float
        Initial allocation per cryptocurrency
    commission : float
        Commission rate per trade
        
    Returns:
    --------
    DataFrame with portfolio performance metrics
    """
    if strategy_params is None:
        strategy_params = {
            'short_window': 5,
            'long_window': 15,
            'roc_threshold': 1.5,
            'rsi_lower': 30,
            'rsi_upper': 70
        }
    
    # Extract parameters
    short_window = strategy_params.get('short_window', 5)
    long_window = strategy_params.get('long_window', 15)
    roc_threshold = strategy_params.get('roc_threshold', 1.5)
    rsi_lower = strategy_params.get('rsi_lower', 30)
    rsi_upper = strategy_params.get('rsi_upper', 70)
    
    # Verify BTC data is available
    if 'btc' not in crypto_data:
        print("Warning: Bitcoin data not found. Cannot use as benchmark.")
        btc_data = None
    else:
        btc_data = crypto_data['btc']
    
    # Find common date range for all cryptocurrencies
    start_dates = []
    end_dates = []
    
    for symbol, data in crypto_data.items():
        if len(data) > max(short_window, long_window) + 10:  # Ensure enough data
            start_dates.append(data.index.min())
            end_dates.append(data.index.max())
    
    common_start = max(start_dates)
    common_end = min(end_dates)
    
    print(f"Common date range: {common_start.date()} to {common_end.date()}")
    
    # Track portfolio performance
    portfolio_value = pd.DataFrame(index=pd.date_range(common_start, common_end))
    portfolio_value['Strategy_Total_Value'] = 0
    portfolio_value['Strategy_Total_Cost'] = 0
    portfolio_value['Buy_Hold_Total_Value'] = 0
    portfolio_value['BTC_Buy_Hold_Value'] = 0
    
    num_cryptos = 0
    successful_cryptos = []
    
    print(f"\nProcessing strategies for all cryptocurrencies...")
    
    # Process each cryptocurrency
    for symbol, data in crypto_data.items():
        try:
            # Filter data to common date range
            filtered_data = data.loc[common_start:common_end]
            
            if len(filtered_data) < max(short_window, long_window) + 10:
                print(f"Skipping {symbol}: insufficient data in common date range")
                continue
            
            # Apply strategy
            strategy_df = simple_momentum_strategy(
                filtered_data,
                short_window=short_window,
                long_window=long_window,
                roc_threshold=roc_threshold,
                rsi_lower=rsi_lower,
                rsi_upper=rsi_upper
            )
            
            # Calculate portfolio allocation
            capital = initial_allocation
            units = 0
            positions = []
            
            # Calculate positions and values over time
            for i in range(len(strategy_df)):
                row = strategy_df.iloc[i]
                
                # If position changed
                if row['Buy']:
                    units = capital * (1 - commission) / row['close']
                    capital = 0
                elif row['Sell']:
                    capital = units * row['close'] * (1 - commission)
                    units = 0
                
                # Record position
                positions.append({
                    'date': strategy_df.index[i],
                    'capital': capital,
                    'units': units,
                    'position_value': units * row['close'],
                    'total_value': capital + (units * row['close'])
                })
            
            # Convert to DataFrame and set index
            position_df = pd.DataFrame(positions)
            position_df.set_index('date', inplace=True)
            
            # Calculate Buy & Hold value
            initial_units = initial_allocation * (1 - commission) / filtered_data['close'].iloc[0]
            buy_hold_values = initial_units * filtered_data['close']
            
            # Add to portfolio total
            portfolio_value = portfolio_value.join(position_df['total_value'], how='left')
            portfolio_value.rename(columns={'total_value': f'{symbol}_value'}, inplace=True)
            
            # Add to aggregate values
            portfolio_value['Strategy_Total_Value'] += position_df['total_value']
            portfolio_value['Strategy_Total_Cost'] += initial_allocation
            portfolio_value['Buy_Hold_Total_Value'] += buy_hold_values
            
            num_cryptos += 1
            successful_cryptos.append(symbol)
            
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    
    # Fill any NaN values with forward fill
    portfolio_value.fillna(method='ffill', inplace=True)
    
    # Calculate BTC Buy & Hold value
    if btc_data is not None:
        btc_filtered = btc_data.loc[common_start:common_end]
        initial_btc_units = (num_cryptos * initial_allocation) * (1 - commission) / btc_filtered['close'].iloc[0]
        portfolio_value['BTC_Buy_Hold_Value'] = initial_btc_units * btc_filtered['close']
    
    # Calculate performance metrics
    portfolio_value['Strategy_Return'] = portfolio_value['Strategy_Total_Value'] / portfolio_value['Strategy_Total_Cost'] - 1
    portfolio_value['Equal_Buy_Hold_Return'] = portfolio_value['Buy_Hold_Total_Value'] / portfolio_value['Strategy_Total_Cost'] - 1
    
    if btc_data is not None:
        portfolio_value['BTC_Buy_Hold_Return'] = portfolio_value['BTC_Buy_Hold_Value'] / portfolio_value['Strategy_Total_Cost'] - 1
    
    # Calculate daily returns for Sharpe ratio
    portfolio_value['Strategy_Daily_Return'] = portfolio_value['Strategy_Total_Value'].pct_change()
    portfolio_value['Equal_Buy_Hold_Daily_Return'] = portfolio_value['Buy_Hold_Total_Value'].pct_change()
    
    if btc_data is not None:
        portfolio_value['BTC_Daily_Return'] = portfolio_value['BTC_Buy_Hold_Value'].pct_change()
    
    # Calculate drawdowns
    portfolio_value['Strategy_DD'] = 1 - portfolio_value['Strategy_Total_Value'] / portfolio_value['Strategy_Total_Value'].cummax()
    portfolio_value['Equal_Buy_Hold_DD'] = 1 - portfolio_value['Buy_Hold_Total_Value'] / portfolio_value['Buy_Hold_Total_Value'].cummax()
    
    if btc_data is not None:
        portfolio_value['BTC_DD'] = 1 - portfolio_value['BTC_Buy_Hold_Value'] / portfolio_value['BTC_Buy_Hold_Value'].cummax()
    
    # Calculate summary metrics
    total_days = (portfolio_value.index[-1] - portfolio_value.index[0]).days
    
    # Strategy metrics
    strategy_return = portfolio_value['Strategy_Return'].iloc[-1]
    strategy_annual_return = (1 + strategy_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    strategy_volatility = portfolio_value['Strategy_Daily_Return'].std() * np.sqrt(365)
    strategy_sharpe = strategy_annual_return / strategy_volatility if strategy_volatility > 0 else 0
    strategy_max_dd = portfolio_value['Strategy_DD'].max()
    
    # Equal-weighted Buy & Hold metrics
    equal_bh_return = portfolio_value['Equal_Buy_Hold_Return'].iloc[-1]
    equal_bh_annual_return = (1 + equal_bh_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    equal_bh_volatility = portfolio_value['Equal_Buy_Hold_Daily_Return'].std() * np.sqrt(365)
    equal_bh_sharpe = equal_bh_annual_return / equal_bh_volatility if equal_bh_volatility > 0 else 0
    equal_bh_max_dd = portfolio_value['Equal_Buy_Hold_DD'].max()
    
    # BTC Buy & Hold metrics
    if btc_data is not None:
        btc_bh_return = portfolio_value['BTC_Buy_Hold_Return'].iloc[-1]
        btc_bh_annual_return = (1 + btc_bh_return) ** (365 / total_days) - 1 if total_days > 0 else 0
        btc_bh_volatility = portfolio_value['BTC_Daily_Return'].std() * np.sqrt(365)
        btc_bh_sharpe = btc_bh_annual_return / btc_bh_volatility if btc_bh_volatility > 0 else 0
        btc_bh_max_dd = portfolio_value['BTC_DD'].max()
    
    # Print summary
    print(f"\nPortfolio Performance Summary:")
    print(f"Number of cryptocurrencies in portfolio: {num_cryptos}")
    print(f"Initial investment: ${num_cryptos * initial_allocation:,.2f}")
    print(f"Final portfolio value (Strategy): ${portfolio_value['Strategy_Total_Value'].iloc[-1]:,.2f}")
    print(f"Final portfolio value (Equal Buy & Hold): ${portfolio_value['Buy_Hold_Total_Value'].iloc[-1]:,.2f}")
    
    if btc_data is not None:
        print(f"Final portfolio value (BTC Buy & Hold): ${portfolio_value['BTC_Buy_Hold_Value'].iloc[-1]:,.2f}")
    
    print("\nPerformance Metrics:")
    print(f"{'Strategy':<20} | {'Total Return':<15} | {'Annual Return':<15} | {'Sharpe Ratio':<15} | {'Max Drawdown':<15}")
    print(f"{'-'*20} | {'-'*15} | {'-'*15} | {'-'*15} | {'-'*15}")
    print(f"{'Momentum Strategy':<20} | {strategy_return:,.2%} | {strategy_annual_return:,.2%} | {strategy_sharpe:.2f} | {strategy_max_dd:,.2%}")
    print(f"{'Equal-Weight B&H':<20} | {equal_bh_return:,.2%} | {equal_bh_annual_return:,.2%} | {equal_bh_sharpe:.2f} | {equal_bh_max_dd:,.2%}")
    
    if btc_data is not None:
        print(f"{'BTC Buy & Hold':<20} | {btc_bh_return:,.2%} | {btc_bh_annual_return:,.2%} | {btc_bh_sharpe:.2f} | {btc_bh_max_dd:,.2%}")
    
    # Create a simple visualization
    plt.figure(figsize=(14, 8))
    
    # Plot portfolio values
    plt.subplot(2, 1, 1)
    plt.plot(portfolio_value.index, portfolio_value['Strategy_Total_Value'], label='Momentum Strategy', linewidth=2)
    plt.plot(portfolio_value.index, portfolio_value['Buy_Hold_Total_Value'], label='Equal-Weight Buy & Hold', linewidth=2)
    
    if btc_data is not None:
        plt.plot(portfolio_value.index, portfolio_value['BTC_Buy_Hold_Value'], label='BTC Buy & Hold', linewidth=2)
    
    plt.title('Aggregate Portfolio Performance')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True)
    plt.legend()
    
    # Plot drawdowns
    plt.subplot(2, 1, 2)
    plt.fill_between(portfolio_value.index, 0, portfolio_value['Strategy_DD'] * 100, label='Strategy DD', color='blue', alpha=0.3)
    plt.fill_between(portfolio_value.index, 0, portfolio_value['Equal_Buy_Hold_DD'] * 100, label='Equal-Weight B&H DD', color='orange', alpha=0.3)
    
    if btc_data is not None:
        plt.fill_between(portfolio_value.index, 0, portfolio_value['BTC_DD'] * 100, label='BTC B&H DD', color='green', alpha=0.3)
    
    plt.title('Portfolio Drawdowns')
    plt.ylabel('Drawdown (%)')
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Prepare summary metrics dictionary
    summary_metrics = {
        'Number of Cryptocurrencies': num_cryptos,
        'Cryptocurrency List': successful_cryptos,
        'Strategy Total Return': strategy_return,
        'Strategy Annual Return': strategy_annual_return,
        'Strategy Sharpe Ratio': strategy_sharpe,
        'Strategy Max Drawdown': strategy_max_dd,
        'Equal-Weight B&H Total Return': equal_bh_return,
        'Equal-Weight B&H Annual Return': equal_bh_annual_return,
        'Equal-Weight B&H Sharpe Ratio': equal_bh_sharpe,
        'Equal-Weight B&H Max Drawdown': equal_bh_max_dd
    }
    
    if btc_data is not None:
        summary_metrics.update({
            'BTC B&H Total Return': btc_bh_return,
            'BTC B&H Annual Return': btc_bh_annual_return,
            'BTC B&H Sharpe Ratio': btc_bh_sharpe,
            'BTC B&H Max Drawdown': btc_bh_max_dd
        })
    
    return portfolio_value, summary_metrics

# Example usage:
portfolio_value, summary_metrics = analyze_aggregate_portfolio_performance(crypto_data)

In [ ]:
def analyze_aggregate_portfolio_performance(crypto_data, strategy_params=None, initial_capital=10000, commission=0.001):
    """
    Analyzes the performance of a portfolio where the capital is equally divided among
    all cryptocurrencies, and each cryptocurrency follows the momentum strategy individually.
    Compares this portfolio to Bitcoin buy & hold strategy.
    
    Parameters:
    -----------
    crypto_data : dict
        Dictionary of cryptocurrency DataFrames
    strategy_params : dict
        Dictionary with strategy parameters
    initial_capital : float
        Total initial capital (will be divided equally among cryptocurrencies)
    commission : float
        Commission rate per trade
        
    Returns:
    --------
    DataFrame with portfolio performance metrics
    """
    if strategy_params is None:
        strategy_params = {
            'short_window': 5,
            'long_window': 15,
            'roc_threshold': 1.5,
            'rsi_lower': 30,
            'rsi_upper': 70
        }
    
    # Extract parameters
    short_window = strategy_params.get('short_window', 5)
    long_window = strategy_params.get('long_window', 15)
    roc_threshold = strategy_params.get('roc_threshold', 1.5)
    rsi_lower = strategy_params.get('rsi_lower', 30)
    rsi_upper = strategy_params.get('rsi_upper', 70)
    
    # Count valid cryptocurrencies for allocation calculation
    valid_cryptos = []
    for symbol, data in crypto_data.items():
        if len(data) > max(short_window, long_window) + 10:  # Ensure enough data
            valid_cryptos.append(symbol)
    
    num_cryptos = len(valid_cryptos)
    print(f"Found {num_cryptos} cryptocurrencies with sufficient data")
    
    # Calculate per-crypto allocation
    if num_cryptos == 0:
        print("Error: No cryptocurrencies with sufficient data found.")
        return None, None
    
    allocation_per_crypto = initial_capital / num_cryptos
    print(f"Allocating ${allocation_per_crypto:.2f} to each cryptocurrency")
    
    # Verify BTC data is available for comparison
    if 'btc' not in crypto_data:
        print("Warning: Bitcoin data not found. Cannot use as benchmark.")
        btc_data = None
    else:
        btc_data = crypto_data['btc']
    
    # Find common date range for all cryptocurrencies
    start_dates = []
    end_dates = []
    
    for symbol in valid_cryptos:
        data = crypto_data[symbol]
        start_dates.append(data.index.min())
        end_dates.append(data.index.max())
    
    common_start = max(start_dates)
    common_end = min(end_dates)
    
    print(f"Common date range: {common_start.date()} to {common_end.date()}")
    
    # Track portfolio performance
    portfolio_value = pd.DataFrame(index=pd.date_range(common_start, common_end))
    portfolio_value['Strategy_Total_Value'] = 0
    portfolio_value['Strategy_Total_Cost'] = 0
    portfolio_value['Equal_Buy_Hold_Total_Value'] = 0
    portfolio_value['BTC_Buy_Hold_Value'] = 0
    
    successful_cryptos = []
    
    print(f"\nProcessing momentum strategies for all cryptocurrencies...")
    
    # Process each cryptocurrency
    for symbol in valid_cryptos:
        data = crypto_data[symbol]
        try:
            # Filter data to common date range
            filtered_data = data.loc[common_start:common_end]
            
            if len(filtered_data) < max(short_window, long_window) + 10:
                print(f"Skipping {symbol}: insufficient data in common date range")
                continue
            
            # Apply momentum strategy
            strategy_df = simple_momentum_strategy(
                filtered_data,
                short_window=short_window,
                long_window=long_window,
                roc_threshold=roc_threshold,
                rsi_lower=rsi_lower,
                rsi_upper=rsi_upper
            )
            
            # Calculate portfolio allocation for this cryptocurrency
            capital = allocation_per_crypto
            units = 0
            positions = []
            
            # Calculate positions and values over time based on strategy signals
            for i in range(len(strategy_df)):
                row = strategy_df.iloc[i]
                
                # If position changed
                if row['Buy']:
                    # Convert cash to crypto
                    units = capital * (1 - commission) / row['close']
                    capital = 0
                elif row['Sell']:
                    # Convert crypto back to cash
                    capital = units * row['close'] * (1 - commission)
                    units = 0
                
                # Record position value for this date
                positions.append({
                    'date': strategy_df.index[i],
                    'capital': capital,
                    'units': units,
                    'position_value': units * row['close'],
                    'total_value': capital + (units * row['close'])
                })
            
            # Convert to DataFrame and set index
            position_df = pd.DataFrame(positions)
            position_df.set_index('date', inplace=True)
            
            # Calculate Buy & Hold value for this cryptocurrency
            initial_units = allocation_per_crypto * (1 - commission) / filtered_data['close'].iloc[0]
            buy_hold_values = initial_units * filtered_data['close']
            
            # Add individual crypto performance to portfolio tracking
            portfolio_value = portfolio_value.join(position_df['total_value'], how='left')
            portfolio_value.rename(columns={'total_value': f'{symbol}_strategy_value'}, inplace=True)
            
            # Also track buy & hold for this crypto
            portfolio_value = portfolio_value.join(pd.Series(buy_hold_values, name=f'{symbol}_bh_value'), how='left')
            
            # Add to aggregate portfolio values
            portfolio_value['Strategy_Total_Value'] += position_df['total_value']
            portfolio_value['Strategy_Total_Cost'] += allocation_per_crypto
            portfolio_value['Equal_Buy_Hold_Total_Value'] += buy_hold_values
            
            successful_cryptos.append(symbol)
            
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    
    # Fill any NaN values with forward fill
    portfolio_value.fillna(method='ffill', inplace=True)
    
    # Calculate BTC Buy & Hold value (using the same total capital)
    if btc_data is not None:
        btc_filtered = btc_data.loc[common_start:common_end]
        initial_btc_units = initial_capital * (1 - commission) / btc_filtered['close'].iloc[0]
        portfolio_value['BTC_Buy_Hold_Value'] = initial_btc_units * btc_filtered['close']
    
    # Calculate performance metrics
    portfolio_value['Strategy_Return'] = portfolio_value['Strategy_Total_Value'] / portfolio_value['Strategy_Total_Cost'] - 1
    portfolio_value['Equal_Buy_Hold_Return'] = portfolio_value['Equal_Buy_Hold_Total_Value'] / portfolio_value['Strategy_Total_Cost'] - 1
    
    if btc_data is not None:
        portfolio_value['BTC_Buy_Hold_Return'] = portfolio_value['BTC_Buy_Hold_Value'] / initial_capital - 1
    
    # Calculate daily returns for Sharpe ratio
    portfolio_value['Strategy_Daily_Return'] = portfolio_value['Strategy_Total_Value'].pct_change()
    portfolio_value['Equal_Buy_Hold_Daily_Return'] = portfolio_value['Equal_Buy_Hold_Total_Value'].pct_change()
    
    if btc_data is not None:
        portfolio_value['BTC_Daily_Return'] = portfolio_value['BTC_Buy_Hold_Value'].pct_change()
    
    # Calculate drawdowns
    portfolio_value['Strategy_DD'] = 1 - portfolio_value['Strategy_Total_Value'] / portfolio_value['Strategy_Total_Value'].cummax()
    portfolio_value['Equal_Buy_Hold_DD'] = 1 - portfolio_value['Equal_Buy_Hold_Total_Value'] / portfolio_value['Equal_Buy_Hold_Total_Value'].cummax()
    
    if btc_data is not None:
        portfolio_value['BTC_DD'] = 1 - portfolio_value['BTC_Buy_Hold_Value'] / portfolio_value['BTC_Buy_Hold_Value'].cummax()
    
    # Calculate summary metrics
    total_days = (portfolio_value.index[-1] - portfolio_value.index[0]).days
    
    # Strategy metrics
    strategy_return = portfolio_value['Strategy_Return'].iloc[-1]
    strategy_annual_return = (1 + strategy_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    strategy_volatility = portfolio_value['Strategy_Daily_Return'].std() * np.sqrt(365)
    strategy_sharpe = strategy_annual_return / strategy_volatility if strategy_volatility > 0 else 0
    strategy_max_dd = portfolio_value['Strategy_DD'].max()
    
    # Equal-weighted Buy & Hold metrics (same cryptos, but buy & hold instead of momentum)
    equal_bh_return = portfolio_value['Equal_Buy_Hold_Return'].iloc[-1]
    equal_bh_annual_return = (1 + equal_bh_return) ** (365 / total_days) - 1 if total_days > 0 else 0
    equal_bh_volatility = portfolio_value['Equal_Buy_Hold_Daily_Return'].std() * np.sqrt(365)
    equal_bh_sharpe = equal_bh_annual_return / equal_bh_volatility if equal_bh_volatility > 0 else 0
    equal_bh_max_dd = portfolio_value['Equal_Buy_Hold_DD'].max()
    
    # BTC Buy & Hold metrics
    if btc_data is not None:
        btc_bh_return = portfolio_value['BTC_Buy_Hold_Return'].iloc[-1]
        btc_bh_annual_return = (1 + btc_bh_return) ** (365 / total_days) - 1 if total_days > 0 else 0
        btc_bh_volatility = portfolio_value['BTC_Daily_Return'].std() * np.sqrt(365)
        btc_bh_sharpe = btc_bh_annual_return / btc_bh_volatility if btc_bh_volatility > 0 else 0
        btc_bh_max_dd = portfolio_value['BTC_DD'].max()
    
    # Print summary
    print(f"\nPortfolio Performance Summary:")
    print(f"Number of cryptocurrencies in portfolio: {len(successful_cryptos)}")
    print(f"Total initial investment: ${initial_capital:,.2f}")
    print(f"Final portfolio value (Momentum Strategy): ${portfolio_value['Strategy_Total_Value'].iloc[-1]:,.2f}")
    print(f"Final portfolio value (Equal Buy & Hold): ${portfolio_value['Equal_Buy_Hold_Total_Value'].iloc[-1]:,.2f}")
    
    if btc_data is not None:
        print(f"Final portfolio value (BTC Buy & Hold): ${portfolio_value['BTC_Buy_Hold_Value'].iloc[-1]:,.2f}")
    
    print("\nPerformance Metrics:")
    print(f"{'Strategy':<20} | {'Total Return':<15} | {'Annual Return':<15} | {'Sharpe Ratio':<15} | {'Max Drawdown':<15}")
    print(f"{'-'*20} | {'-'*15} | {'-'*15} | {'-'*15} | {'-'*15}")
    print(f"{'Momentum Strategy':<20} | {strategy_return:,.2%} | {strategy_annual_return:,.2%} | {strategy_sharpe:.2f} | {strategy_max_dd:,.2%}")
    print(f"{'Equal-Weight B&H':<20} | {equal_bh_return:,.2%} | {equal_bh_annual_return:,.2%} | {equal_bh_sharpe:.2f} | {equal_bh_max_dd:,.2%}")
    
    if btc_data is not None:
        print(f"{'BTC Buy & Hold':<20} | {btc_bh_return:,.2%} | {btc_bh_annual_return:,.2%} | {btc_bh_sharpe:.2f} | {btc_bh_max_dd:,.2%}")
    
    # Create a simple visualization
    plt.figure(figsize=(14, 8))
    
    # Plot portfolio values
    plt.subplot(2, 1, 1)
    plt.plot(portfolio_value.index, portfolio_value['Strategy_Total_Value'], label='Momentum Strategy', linewidth=2)
    plt.plot(portfolio_value.index, portfolio_value['Equal_Buy_Hold_Total_Value'], label='Equal-Weight Buy & Hold', linewidth=2)
    
    if btc_data is not None:
        plt.plot(portfolio_value.index, portfolio_value['BTC_Buy_Hold_Value'], label='BTC Buy & Hold', linewidth=2)
    
    plt.title('Aggregate Portfolio Performance')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True)
    plt.legend()
    
    # Plot drawdowns
    plt.subplot(2, 1, 2)
    plt.fill_between(portfolio_value.index, 0, portfolio_value['Strategy_DD'] * 100, label='Strategy DD', color='blue', alpha=0.3)
    plt.fill_between(portfolio_value.index, 0, portfolio_value['Equal_Buy_Hold_DD'] * 100, label='Equal-Weight B&H DD', color='orange', alpha=0.3)
    
    if btc_data is not None:
        plt.fill_between(portfolio_value.index, 0, portfolio_value['BTC_DD'] * 100, label='BTC B&H DD', color='green', alpha=0.3)
    
    plt.title('Portfolio Drawdowns')
    plt.ylabel('Drawdown (%)')
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Calculate performance attribution - which cryptos contributed most to returns
    performance_attribution = []
    for symbol in successful_cryptos:
        if f'{symbol}_strategy_value' in portfolio_value.columns:
            initial_value = allocation_per_crypto
            final_value = portfolio_value[f'{symbol}_strategy_value'].iloc[-1]
            total_return = (final_value / initial_value) - 1
            contribution = (final_value - initial_value) / initial_capital
            
            performance_attribution.append({
                'symbol': symbol,
                'initial_value': initial_value,
                'final_value': final_value,
                'return': total_return,
                'contribution': contribution
            })
    
    attribution_df = pd.DataFrame(performance_attribution)
    if not attribution_df.empty:
        attribution_df = attribution_df.sort_values('contribution', ascending=False)
        
        print("\nTop 5 Contributors to Portfolio Return:")
        print(attribution_df.head(5)[['symbol', 'return', 'contribution']])
        
        print("\nBottom 5 Contributors to Portfolio Return:")
        print(attribution_df.tail(5)[['symbol', 'return', 'contribution']])
    
    # Prepare summary metrics dictionary
    summary_metrics = {
        'Number of Cryptocurrencies': len(successful_cryptos),
        'Cryptocurrency List': successful_cryptos,
        'Strategy Total Return': strategy_return,
        'Strategy Annual Return': strategy_annual_return,
        'Strategy Sharpe Ratio': strategy_sharpe,
        'Strategy Max Drawdown': strategy_max_dd,
        'Equal-Weight B&H Total Return': equal_bh_return,
        'Equal-Weight B&H Annual Return': equal_bh_annual_return,
        'Equal-Weight B&H Sharpe Ratio': equal_bh_sharpe,
        'Equal-Weight B&H Max Drawdown': equal_bh_max_dd
    }
    
    if btc_data is not None:
        summary_metrics.update({
            'BTC B&H Total Return': btc_bh_return,
            'BTC B&H Annual Return': btc_bh_annual_return,
            'BTC B&H Sharpe Ratio': btc_bh_sharpe,
            'BTC B&H Max Drawdown': btc_bh_max_dd
        })
    
    if not attribution_df.empty:
        summary_metrics['Performance Attribution'] = attribution_df
    
    return portfolio_value, summary_metrics


In [ ]:
def main():
    # Load all data
    crypto_data = load_all_data()
    
    # Run the basic strategy on all cryptos
    multi_crypto_results = run_strategy_on_all_cryptos(crypto_data)
    print("\nIndividual cryptocurrency performance summary:")
    print(multi_crypto_results['summary'][['symbol', 'total_return', 'sharpe_ratio', 'alpha_vs_btc']].head(10))
    
    # Run the aggregate portfolio analysis
    print("\n\n=========== PORTFOLIO LEVEL ANALYSIS ===========")
    portfolio_data, portfolio_metrics = analyze_aggregate_portfolio_performance(crypto_data)
    
    # You can now access the portfolio_data DataFrame for further analysis
    # or use the portfolio_metrics dictionary for reporting

if __name__ == "__main__":
    main()